In [227]:
import torch 
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import genfromtxt
%matplotlib inline

In [235]:
train = np.loadtxt("Blocks/TrainData1.txt",skiprows=0)


test = np.loadtxt("Blocks/TestData1.txt",skiprows=0)


In [238]:
print(train)
print(test)
print(type(train))

[ 1.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
[14. 15. 16. 17. 18. 19. 20.]
<class 'numpy.ndarray'>


In [283]:
tensor_train = torch.from_numpy(train)
tensor_test = torch.from_numpy(test)
tensor_train = torch.FloatTensor(train)
tensor_test = torch.FloatTensor(test)



<class 'torch.Tensor'>


In [273]:
def input_data(seq,ws):
    out = []
    L=len(seq)

    for i in range(L-ws):
        window = seq[i:i+ws]
        label = seq[i+ws:i+ws+1]
        out.append((window,label))
    return out

window_size = 10

train_list = input_data(tensor_train, window_size)
print(train_list)


<class 'list'>
[(tensor([1., 1., 2., 3., 4., 5., 6., 7., 8., 9.]), tensor([10.])), (tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]), tensor([11.])), (tensor([ 2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.]), tensor([12.])), (tensor([ 3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.]), tensor([13.]))]


In [242]:
class LSTM(nn.Module):
    def __init__(self,input_size=1,hidden_size=50,out_size=1 ):
    
        super().__init__()
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size,hidden_size)
        self.linear = nn.Linear(hidden_size,out_size)
        
        self.hidden = (torch.zeros(1,1,hidden_size), torch.zeros(1,1,hidden_size))
    def forward(self, seq):
        
        lstm_out , self.hidden = self.lstm(seq.view(len(seq),1,-1),self.hidden)
        pred = self.linear(lstm_out.view(len(seq),-1))
        
        return pred[-1]
    
        

In [243]:
torch.manual_seed(42)
model = LSTM()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)
model

LSTM(
  (lstm): LSTM(1, 50)
  (linear): Linear(in_features=50, out_features=1, bias=True)
)

In [244]:
for p in model.parameters():
    print(p.numel())

200
10000
200
200
50
1


In [280]:
train_data

array([[ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13]], dtype=int64)

In [285]:
epochs=3
future=40

for i in range(epochs):
    for seq,y_train in train_list:
        
        optimizer.zero_grad()
        model.hidden = (torch.zeros(1,1,model.hidden_size),
                        torch.zeros(1,1,model.hidden_size))
        
        y_pred = model(seq)
        
        loss = criterion(y_pred,y_train)
        loss.backward()
        optimizer.step()
        
        
    print(f"Epoch {i} Loss: {loss.item()}")
    preds = train[-window_size:].tolist()
    
    for f in range(future):
        
        seq = torch.FloatTensor(preds[-window_size:])
        
        with torch.no_grad():
            
            model.hidden = (torch.zeros(1,1,model.hidden_size), 
                            torch.zeros(1,1,model.hidden_size))
            preds.append(model(seq).item())
            
    loss = criterion(torch.tensor(preds[-window_size:]),train_tensor[12:])
    print(f'Performance on test range: {loss}')
    
    plt.figure(figsize=(12,4))
    plt.xlim=(700,801)
    plt.grid(True)
    plt.plot(y.numpy())
    plt.plot(range(760,800),preds[window_size:])
    plt.show()

Epoch 0 Loss: 0.900484025478363


RuntimeError: Expected object of scalar type Float but got scalar type Long for argument #2 'target'